Dependencies

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pickle

Kamera Videoaufnahme

In [2]:
import time
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from collections import Counter
import pickle
import threading

# 1. Modell laden
model = load_model('/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/backend_fastapi/ai_models/2DCNN_Versuch9_BiggerBagging_M1.keras')

with open('2DCNN_Versuch9_BiggerBagging_M_Labelencoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# 3. VideoCapture-Objekt erstellen
cap = cv2.VideoCapture(0)  # 0 ist normalerweise die Standard-Webcam

num_frames = 16
frames = []
predictions = []

frame_count = 0  # Zähler für die Anzahl der verarbeiteten Frames
fps_values = []  # Liste zur Speicherung der FPS-Werte
last_time = time.time()  # Zeitpunkt zu Beginn
target_fps = 16  # Ziel-FPS
frame_time = 1 / target_fps  # Zeit, die jedes Frame benötigen sollte (in Sekunden)
last_fps = 0

# Queue für das Threading, um Vorhersagen im Hintergrund zu verarbeiten
from queue import Queue
prediction_queue = Queue()

# Funktion für die Vorhersage im Hintergrund
def predict_in_background(frames_input):
    predictions = model.predict(frames_input)
    predicted_class = np.argmax(predictions, axis=1)
    class_label = encoder.inverse_transform(predicted_class)[0]
    # Das Ergebnis in die Queue legen, damit der Hauptthread es abholen kann
    prediction_queue.put(class_label)

while True:
    # Zeitstempel vor dem Frame-Lesen
    frame_start_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break

    # Frame vorverarbeiten
    input_size = (224, 224)  # Die Größe, die dein Modell erwartet
    frame_resized = cv2.resize(frame, input_size)
    frame_normalized = frame_resized / 255.0  # Normalisieren

    # Hinzufügen des vorverarbeiteten Frames zur Liste
    frames.append(frame_normalized)

    # Wenn 16 Frames gesammelt wurden, führe eine Vorhersage durch
    if len(frames) == num_frames:
        # Umwandeln in ein NumPy-Array und Hinzufügen einer Batch-Dimension
        frames_input = np.array(frames)
        frames_input = np.expand_dims(frames_input, axis=0)  # (1, 16, 224, 224, 3)

        # Vorhersage im Hintergrund starten (im separaten Thread)
        prediction_thread = threading.Thread(target=predict_in_background, args=(frames_input,))
        prediction_thread.start()

        # Die Liste der Frames zurücksetzen, um neue Frames zu sammeln
        frames.pop(0)

    # Wenn Vorhersage abgeschlossen ist, das Ergebnis holen
    if not prediction_queue.empty():
        class_label = prediction_queue.get()
        print(f"Predicted: {class_label}")

        predictions.append(class_label)
        most_common = Counter(predictions).most_common(1)[0]

        # Text auf das Frame zeichnen
        cv2.putText(frame, "Sequence predition: " + most_common , (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5)
        cv2.putText(frame, "Frame predition: " + class_label , (10, 250), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 255, 0), 5)
        cv2.putText(frame, "FPS: " + str(last_fps) + "fps", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 5)

    # Zeit nach der Verarbeitung des Frames
    frame_end_time = time.time()
    frame_processing_time = frame_end_time - frame_start_time

    # Berechnung der verbleibenden Zeit, um auf das Ziel FPS zu kommen
    remaining_time = frame_time - frame_processing_time

    if remaining_time > 0:
        # Warten, bis das Ziel-FPS erreicht wird
        time.sleep(remaining_time)

    # FPS-Berechnung und Ausgabe
    current_time = time.time()
    elapsed_prediction_time = current_time - last_time
    fps = 1 / elapsed_prediction_time
    last_fps = round(fps, 2)
    
    last_time = current_time
    print("Camera FPS: " + str(fps) + " Elapsed Time is: " + str(elapsed_prediction_time))

    # Frame anzeigen
    cv2.imshow('Webcam für Livepredction', frame)

    # Wenn 'q' gedrückt wird, stoppe die Schleife
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Camera FPS: 15.299805574503631 Elapsed Time is: 0.06536030769348145
Camera FPS: 7.050447218771589 Elapsed Time is: 0.14183497428894043
Camera FPS: 11.889762618846488 Elapsed Time is: 0.0841059684753418
Camera FPS: 11.765822678283897 Elapsed Time is: 0.0849919319152832
Camera FPS: 12.043668774299514 Elapsed Time is: 0.08303117752075195


2024-12-13 08:04:40.045 Python[55547:721941] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-13 08:04:40.045 Python[55547:721941] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Camera FPS: 9.962764668715765 Elapsed Time is: 0.10037374496459961
Camera FPS: 11.597143228439421 Elapsed Time is: 0.0862281322479248
Camera FPS: 12.243640715766121 Elapsed Time is: 0.08167505264282227
Camera FPS: 12.184597507480463 Elapsed Time is: 0.08207082748413086
Camera FPS: 12.12931211484128 Elapsed Time is: 0.08244490623474121
Camera FPS: 11.89569755239797 Elapsed Time is: 0.08406400680541992
Camera FPS: 11.51756067287995 Elapsed Time is: 0.08682394027709961
Camera FPS: 11.424263223838318 Elapsed Time is: 0.08753299713134766
Camera FPS: 11.283321586329716 Elapsed Time is: 0.08862638473510742
Camera FPS: 11.28784612610576 Elapsed Time is: 0.08859086036682129
Camera FPS: 11.305828252279062 Elapsed Time is: 0.08844995498657227
Camera FPS: 11.859304666470628 Elapsed Time is: 0.08432197570800781
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Predicted: nothing_happening
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Camera FPS: 12.14933855498814 Elapsed Time is: 0.08230900764465332
1/1 ━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 757ms/step


: 